In [ ]:
import collections
import glob
import json
import openreview
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns


dataset_dir = "data_prep/final_dataset/"

SUBSETS = "train dev test".split()

datasets = collections.defaultdict(list)

for subset in SUBSETS:
    for filename in glob.glob(dataset_dir + subset + "/*"):
        with open(filename, 'r') as f:
            datasets[subset].append(json.load(f))

In [ ]:
def total_and_average_len(list_of_lists):
    #print(list_of_lists)
    big_list = sum(list_of_lists, [])
    return len(big_list), len(big_list)/len(list_of_lists)

def count_dataset(pairs, subset):
    # TODO: Add double-annotated and adjudicated
    review_total, review_average = total_and_average_len([pair["review_sentences"] for pair in pairs])
    rebuttal_total, rebuttal_average = total_and_average_len([pair["rebuttal_sentences"] for pair in pairs])
    return {
        "subset":subset,
        "pairs": len(pairs),
        "forums": len(set(pair["metadata"]["forum_id"] for pair in pairs)),
        "adjudicated": len([pair for pair in pairs if pair["metadata"]["annotator"] == "anno0"]),
        "review_sentences": review_total,
        "rebuttal_sentences": rebuttal_total,
        "review_avg_sentences": review_average,
        "rebuttal_avg_sentences": rebuttal_average,
        
    }
# Distribution of examples over sets
df_dicts = [count_dataset(pairs, subset) for subset, pairs in datasets.items()]
dataframe = pd.DataFrame.from_dict(df_dicts)

dataframe.round(2).transpose()

# Appendices

In [ ]:
# Review and rebuttal length

big_list = sum(datasets.values(), [])

length_counter = collections.Counter()

for example in big_list:
    length_counter[(len(example["review_sentences"]), 
                    len(example["rebuttal_sentences"]))] += 1
df = pd.DataFrame.from_dict([
    {
        "review_length": a,
        "rebuttal_length": b,
        "count": count
    } for (a, b), count in length_counter.items()
])
sns.jointplot(x=df.review_length, y=df.rebuttal_length, cmap="Blues", kind='hist')
plt.show()

In [ ]:
import tqdm
guest_client = openreview.Client(baseurl='https://api.openreview.net')

def get_num_reviews(guest_client, forum_id):
    forum_notes = guest_client.get_notes(forum=forum_id)
    review_count = 0
    for note in forum_notes:
        if note.replyto == forum_id and 'Reviewer' in note.signatures[0]:
            review_count += 1
    return review_count

# for subset, examples in datasets.items():
#     forum_counter = collections.defaultdict(list)
#     for example in tqdm.tqdm(examples[:100]):
#         forum_counter[example["metadata"]["forum_id"]].append(example["metadata"]["review_id"])
    
    
#     percentage_annotated_list = []

#     for forum, annotated_reviews in tqdm.tqdm(forum_counter.items()):
#         percentage_annotated_list.append(len(annotated_reviews)/get_num_reviews(guest_client, forum))
        
#     sns.histplot(data=percentage_annotated_list)
#     break


In [ ]:
# Annotator confusion
